In [1]:
import pandas as pd
import numpy as np


In [2]:
data_set = pd.read_csv('datasets/customer_data.csv')
data_set.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data_set['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [5]:
data_set['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [9]:
male = pd.get_dummies(data_set['Gender'], drop_first=True)


In [24]:
X = data_set.iloc[:, 3:13]
Y = data_set.iloc[:, 13]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [25]:
X.drop(columns=['Gender', 'Geography'], inplace =True)

In [26]:
X = pd.concat([X,male], axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male
0,619,42,2,0.00,1,1,1,101348.88,0
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,0
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0


In [27]:
Y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=42, test_size=0.2)

In [29]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [34]:
X_train.shape

(8000, 9)

### optimization packages

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [44]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Embedding,LeakyReLU,BatchNormalization,Dropout
from keras.activations import relu, sigmoid

In [36]:
def create_model(layers, activation):
    model= Sequential()
    for i, nodes in enumerate(layers):
        if i ==0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [37]:
model= KerasClassifier(build_fn= create_model, verbose=0)

In [38]:
layers = [[20], [40,20],[45,30,15]]
activations = ['sigmoid', 'relu']

In [39]:
param_grid = dict(layers= layers, activation = activations, batch_size= [128,256], epochs=[30])
param_grid

{'layers': [[20], [40, 20], [45, 30, 15]],
 'activation': ['sigmoid', 'relu'],
 'batch_size': [128, 256],
 'epochs': [30]}

In [40]:
grid = GridSearchCV(estimator = model, param_grid= param_grid, cv=5, n_jobs=-1)
#cross validate(cv) 5 times to get 5 different scores

In [41]:
grid

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001FAADE4DE80>,
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [128, 256], 'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15]]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [45]:
grid_result = grid.fit(X_train, Y_train)

In [48]:
print(grid_result.best_score_, grid_result.best_params_)

0.8535000085830688 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}


In [53]:
grid_result.best_score_


0.8535000085830688

In [56]:
grid

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001FAADE4DE80>,
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [128, 256], 'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15]]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

### predicting test data

In [65]:
y_pred = (grid.predict(X_test) > 0.5 ).astype('int32')
y_pred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [66]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, Y_test)
print(score * 100)

85.85000000000001
